For text to speech model



In [ ]:
#installing library


In [ ]:
#we require two libraries for this part
#one is gtts to convert text to speech

In [ ]:
#other is googletrans to convert language as per our need

In [ ]:
! pip install gtts

In [ ]:
! pip install googletrans

In [ ]:
#importing gTTS from gtts

In [ ]:
from gtts import gTTS

In [ ]:
from googletrans import Translator

In [ ]:
translator = Translator()

for speech to text model


In [ ]:
#installing libraries

In [ ]:
! pip install gtts

In [ ]:
! pip install SpeechRecognition

In [ ]:
import speech_recognition as sr

In [ ]:
r = sr.Recognizer()

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1).
libportaudiocpp0 is already the newest version (19.6.0-1).
portaudio19-dev is already the newest version (19.6.0-1).
libasound2-dev is already the newest version (1.1.3-5ubuntu0.6).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [ ]:
! pip install pyaudio

In [ ]:
# all imports
from io import BytesIO
from base64 import b64decode
from google.colab import output
from IPython.display import Javascript

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  print("Speak Now...")
  display(Javascript(RECORD))
  sec += 1
  s = output.eval_js('record(%d)' % (sec*1000))
  print("Done Recording !")
  b = b64decode(s.split(',')[1])
  return b #byte stream

In [ ]:
audio = record(5)   #will record for 5 seconds

Speak Now...


<IPython.core.display.Javascript object>

Done Recording !


In [ ]:
type(audio)

bytes

In [ ]:
import IPython.display as ipd

ipd.display(ipd.Audio(audio))

In [ ]:
#try:
    #r.recognize_google(audio)
#except:
    #print("try again:")

In [57]:
import keras
import PIL
import numpy as np
import cv2
import random
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout

In [59]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [60]:
classifier=Sequential()

In [61]:
# step 1: convolution
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))

In [62]:
# step 2:pooling
classifier.add(MaxPooling2D(pool_size=(2,2)))

# adding another convolutional layer to imoprove the accuracy
classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size= (2,2)))

In [63]:
#step 3:flattening
classifier.add(Flatten())

In [64]:
# step 4: full connection
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(0.6))
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dropout(0.3))
classifier.add(Dense(units=1, activation='sigmoid'))
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               802944    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0

In [65]:
# compiling
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [66]:
#fit cnn to images
from keras.preprocessing.image import ImageDataGenerator
training_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
testing_datagen = ImageDataGenerator(rescale=1./255)

training_set = training_datagen.flow_from_directory('/content/gdrive/My Drive/dataset', 
                                                    target_size=(64,64), 
                                                    batch_size=32, 
                                                    class_mode='binary')

Found 1396 images belonging to 2 classes.


In [67]:
test_set = testing_datagen.flow_from_directory('/content/gdrive/My Drive/test_set',
                                                target_size=(64,64), 
                                                batch_size=32,
                                                class_mode='binary')

Found 42 images belonging to 2 classes.


In [68]:
classifier.fit_generator(training_set, 
                         steps_per_epoch=100/8, 
                         epochs=8, 
                         validation_data=test_set, 
                         validation_steps=8/8)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/8
12/12 [==============================] - 219s 18s/step - loss: 0.6901 - accuracy: 0.5495 - val_loss: 0.5716 - val_accuracy: 0.6875
Epoch 2/8
12/12 [==============================] - 137s 11s/step - loss: 0.5422 - accuracy: 0.7548 - val_loss: 0.3520 - val_accuracy: 0.9688
Epoch 3/8
12/12 [==============================] - 89s 7s/step - loss: 0.3434 - accuracy: 0.8688 - val_loss: 0.1820 - val_accuracy: 0.9375
Epoch 4/8
12/12 [==============================] - 62s 5s/step - loss: 0.2662 - accuracy: 0.9231 - val_loss: 0.1305 - val_accuracy: 0.9688
Epoch 5/8
12/12 [==============================] - 47s 4s/step - loss: 0.1600 - accuracy: 0.9543 - val_loss: 0.0944 - val_accuracy: 0.9375
Epoch 6/8
12/12 [==============================] - 38s 3s/step - loss: 0.1868 - accuracy: 0.9255 - val_loss: 0.1406 - val_accuracy: 0.9062
Epoch 7/8
12/12 [==============================] - 25s 2s/step - loss: 0.1322 - accuracy: 0.9591 - val_loss: 0.0168 - val_accuracy: 1.0000
Epoch 8/8
12/12 [======

In [69]:
test_pred = classifier.predict(test_set)>0.5
test_pred

array([[ True],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [False]])

In [72]:
print("this is a design of a multipurpose ai model")
print("welcome to the menu")
print("do yo want to use the multipurpose model for")
d=input("press y for yes and n for no")
print("1.speech to text ")
print("2.text to speech")
print("3.face mask detector")
print("4.exit")
while d=='y':
    c=input("enter your choice")
    if int(c)==1:
        text="hi there how you doing"
        language='en'
        myobj = gTTS(text=text, lang=language , slow = False)
        myobj.save("demo.mp3")
        #break    
    elif int(c)==2:
        audio = record(5)
        type(audio)
        import IPython.display as ipd
        ipd.display(ipd.Audio(audio))
        try:
          text= r.recognize_google(audio)
          print('you said:{}'.format(text))
        except:
          print("please try again")
        #break
    elif int(c)==3:
        # predictions for with_mask image using train model
        import numpy as np
        from keras.preprocessing import image
        tested_image = image.load_img('/content/gdrive/My Drive/test_set/test _mask/11-with-mask.jpg',target_size=(64,64))
        tested_image = image.img_to_array(tested_image)
        tested_image = np.expand_dims(tested_image,axis = 0)
        output = classifier.predict(tested_image)
        training_set.class_indices
        if output[0][0] ==1:
          prediction = 'without mask'
          print(prediction )
        else:
          prediction = 'with mask'
          print(prediction )
          import numpy as np
          from keras.preprocessing import image
          tested_image = image.load_img('/content/gdrive/My Drive/test_set/test_withoutmask/54.jpg',target_size=(64,64))
          tested_image = image.img_to_array(tested_image)
          tested_image = np.expand_dims(tested_image,axis = 0)
          output = classifier.predict(tested_image)
          training_set.class_indices
          if output[0][0] ==1:
            prediction = 'without mask'
            print(prediction )
          else:
            prediction = 'with mask'
            print(prediction )
        #break    
    elif int(c)==4:
        print("exit")
    else:
        print("invalid")
    d=input("enter ur choice y/n")
    if d=='n':
        print("process ends here")
        break
    else:
        pass
        

this is a design of a multipurpose ai model
welcome to the menu
do yo want to use the multipurpose model for
press y for yes and n for noy
1.speech to text 
2.text to speech
3.face mask detector
4.exit
enter your choice3
with mask
without mask
enter ur choice y/nn
process ends here
